# Convert ePRTR data to CLRTAP LPS submission
Germany's thru.de website offers the German ePRTR dataset as a SQLITE database. In this script, I convert the data given to the LRTAP convention's Excel template format. On the way, some information is augmented from other sources, in particular stack heights and GNFR.

In [ ]:
import sqlite3 as sql
import pandas as pd
import csv
import codecs

### 1. Load and transform database content 

Let's start and connect to the database as downloaded from https://www.thru.de/thrude/downloads/

The database contains a couple of tables, but we are mainly interested in `facilities`, `activities` and `releases`. The `facilities` table already has the list of point sources we need and offers some properties right away. We can simply grap the names and coordinates, for example. As we only need the most current data, we will also filter on the `year` column.

Next, we need to match the point source's category (`prtr_id`) and the emissions from the `activities` and `releases` tables respectivly. Note that `releases` has more data than just emissions, so we need to filter for the correct `compartment`.

The query to achieve these two steps looks something like this:
```
SELECT name, prtr_key, wgs84_x, wgs84_y, substance_name, annual_load, [...]
FROM facilities
INNER JOIN activities ON facilities.id = activities.facility_id
INNER JOIN releases ON facilities.id = releases.facility_id
WHERE facilities.year = <year> and releases.compartment = 'Air')
```

This query will deliver a list of point sources and their releases, but there is still one row per pollutant. To get closer to our CLRTAP template, we need to transpose these emissions to columns and groups them by point source.

```
SELECT
	name,
	prtr_key AS 'GNFR (TODO)',
	administrative_number AS 'E-PRTR ID',
	'' AS 'Height class (TODO)',
	wgs84_x, wgs84_y,
	SUM(annual_load / 1000000) FILTER (WHERE substance_name = 'Nitrogen oxides (NOx/NO2)') AS 'NOx (as NO2) [kt]',
	SUM(annual_load / 1000000) FILTER (WHERE substance_name = 'Sulphur oxides (SOx/SO2)') AS 'SOx (as SO2) [kt]',
    [...]
FROM ( <inner query> )
GROUP BY id
ORDER BY name
```

Before we go ahead and test all this out, let's list a few underlying assumptions:

#### Database content pre-conditions
There are a few things we take for granted when working with the data source, including:
* All release annual load values are given in [kg]
* facilities.year == release.year for all joins on facilities.id
* All substance names are correctly register, no misspellings etc.

Okay, with this out of the way, let's go and try this...

In [ ]:
connection = sql.connect('../source/prtr_en.db')

sql_string = open('../sql/all lps one year.sql', mode='r', encoding='utf-8-sig').read()
data = pd.read_sql_query(sql_string, connection)

connection.close()

data.info(verbose=True)
data.columns

In [ ]:
data.sort_values('NOx (as NO2) [kt]', ascending=False).head(10)

In [ ]:
data.sort_values('SOx (as SO2) [kt]', ascending=False).head(10)

In [ ]:
data.sort_values('NH3 [kt]', ascending=False).head(10)

### 2. Add GNFR and stack height information

First, we need to map the PRTR activities to their GNFR equivalents.
We do the same for the stack heights using the UBA Report FKZ:3717511010.

In [ ]:
gnfr_mapping = {}
with open('../source/GNFR mapping neo filled.csv', newline='') as mapping_file: 
    for line in csv.reader(mapping_file, delimiter=';'): 
        gnfr_mapping[line[0]] = line[2]
stack_mapping={}
with open('../source/GNFR mapping neo filled.csv', newline='') as mapping_file: 
    for line in csv.reader(mapping_file, delimiter=';'): 
        stack_mapping[line[0]] = line[3]
data.head(10)

In [ ]:
data.replace({'GNFR (TODO)': gnfr_mapping}, inplace=True)
data.replace({'Height class (TODO)': stack_mapping}, inplace=True)
data.rename(columns={'GNFR (TODO': 'GNFR'}, inplace=True)
data.rename(columns={'Height class (TODO)' : 'Height class'}, inplace=True)
data.head(10)

### 3. Write result out as csv (to be copied to the Excel template)

In [ ]:
data.drop(columns=['PRTR activity (DELETE!)', 'NACE (DELETE!)'], inplace=True)
data.to_csv('../target/output.csv', sep=';', index=False)